## BDD

In [1]:
pip install PyPDF2 pandas scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from PyPDF2 import PdfReader
import pandas as pd

# Fonction pour extraire le texte d'un PDF
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:  # Vérifier si l'extraction a réussi
                text += page_text + "\n"
        return text.strip()  # Supprime les espaces inutiles
    except Exception as e:
        return f"Error reading {pdf_path}: {str(e)}"

# Dossier contenant les PDF
directory = "BDD"
data = []

# Vérifier si le dossier existe
if os.path.exists(directory):
    # Parcourir tous les fichiers du dossier
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):  # Vérifier que c'est un fichier PDF
            pdf_path = os.path.join(directory, filename)
            extracted_text = extract_text_from_pdf(pdf_path)
            data.append([filename, extracted_text])  # Ajouter aux données

    # Créer un DataFrame
    pdf_dataframe = pd.DataFrame(data, columns=["Title", "Content"])

    # Nettoyer les noms des fichiers
    pdf_dataframe['Title'] = pdf_dataframe['Title'].str.replace(r'\d+', '', regex=True)  # Supprimer les nombres
    pdf_dataframe['Title'] = pdf_dataframe['Title'].str.replace(r'\.pdf$', '', regex=True)  # Supprimer ".pdf"
    pdf_dataframe['Title'] = pdf_dataframe['Title'].str.replace(r'[_-]', ' ', regex=True)  # Remplacer "_" et "-" par des espaces

else:
    print(f"Le dossier '{directory}' n'existe pas.")


In [3]:
print("Valeurs uniques du titre :")
print(pdf_dataframe['Title'].unique()[:100])

Valeurs uniques du titre :
['assignation  absenceNomDemandeur' 'assignation  absenceNomTribunal'
 'requete  absenceNomOpposant' 'requete  absenceDateNaissanceDemandeur'
 'requete  absenceNomDemandeur'
 'assignation  absenceLieuNaissanceDemandeur'
 'requete  absenceLieuNaissanceDemandeur' 'assignation  sansVices'
 'requete  absenceDateDepot' 'requete  absenceNomChambre'
 'assignation  absenceNomHuissier' 'assignation  absenceNomChambre'
 'assignation  absenceNomAvocat' 'requete  absenceLieuRésidenceDemandeur'
 'assignation  absenceDateHeureAudience' 'assignation  absenceSignature'
 'requete  absenceProfessionDemandeur' 'assignation  absenceNomAssigné'
 'assignation  absenceDateNaissanceDemandeur' 'assignation  absenceAnnée'
 'requete  absenceNomTribunal' 'requete  absenceLieuRésidenceOpposant'
 'assignation  absenceDateAudience' 'requete  absenceSignature'
 'assignation  absenceHeureAudience' 'requete  sansVices'
 'assignation  absencedateAudience']


In [4]:
title_mapping = {
    'assignation  absenceNomDemandeur': 0,
    'assignation  absenceNomTribunal': 1,
    'requete  absenceNomOpposant': 2,
    'requete  absenceDateNaissanceDemandeur': 3,
    'requete  absenceNomDemandeur': 4,
    'assignation  absenceLieuNaissanceDemandeur': 5,
    'requete  absenceLieuNaissanceDemandeur': 6,
    'assignation  sansVices': 7,
    'requete  absenceDateDepot': 8,
    'requete  absenceNomChambre': 9,
    'assignation  absenceNomHuissier': 10,
    'assignation  absenceNomChambre': 11,
    'assignation  absenceNomAvocat': 12,
    'requete  absenceLieuRésidenceDemandeur': 13,
    'assignation  absenceDateHeureAudience': 14,
    'assignation  absenceSignature': 15,
    'requete  absenceProfessionDemandeur': 16,
    'assignation  absenceNomAssigné': 17,
    'assignation  absenceDateNaissanceDemandeur': 18,
    'assignation  absenceAnnée': 19,
    'requete  absenceNomTribunal': 20,
    'requete  absenceLieuRésidenceOpposant': 21,
    'assignation  absenceDateAudience': 22,
    'requete  absenceSignature': 23,
    'assignation  absenceHeureAudience': 24,
    'requete  sansVices': 25,
    'assignation  absencedateAudience': 26
}

# Appliquer le mapping sur les titres
pdf_dataframe['Title'] = pdf_dataframe['Title'].replace(title_mapping)

/var/folders/_j/41f8hxv911z0jxlfypdr3xrw0000gn/T/ipykernel_63564/1345455003.py:32: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pdf_dataframe['Title'] = pdf_dataframe['Title'].replace(title_mapping)


In [5]:
print("Valeurs uniques du titre :")
print(pdf_dataframe['Title'].unique()[:100])

Valeurs uniques du titre :
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]


In [6]:
class_distribution = pdf_dataframe['Title'].value_counts()
# Afficher uniquement les premières valeurs de la distribution des classes
print("Distribution des classes avant suréchantillonnage :")
print(class_distribution.head())


Distribution des classes avant suréchantillonnage :
Title
7     31
0     10
25    10
8     10
12     9
Name: count, dtype: int64


In [7]:
from sklearn.utils import resample

# Vérifier la distribution des classes avant le suréchantillonnage
class_distribution = pdf_dataframe['Title'].value_counts()
print("Distribution des classes avant suréchantillonnage :")
print(class_distribution.head())

# Définir le seuil minimal de valeurs par classe
min_samples = 32

# Liste pour stocker les nouvelles données après suréchantillonnage
resampled_data = []

for class_label, count in class_distribution.items():
    class_df = pdf_dataframe[pdf_dataframe['Title'] == class_label]
    
    # Si la classe a moins de min_samples, on la suréchantillonne
    if count < min_samples:
        resampled_class_df = resample(class_df, replace=True, n_samples=min_samples, random_state=42)
    else:
        resampled_class_df = class_df  # Conserver la classe telle quelle si elle a au moins min_samples
    
    resampled_data.append(resampled_class_df)

# Concaténer toutes les classes suréchantillonnées
resampled_df = pd.concat(resampled_data, ignore_index=True)

# Mélanger les données après le suréchantillonnage
resampled_df = resampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Vérifier la distribution des classes après suréchantillonnage
class_distribution_after = resampled_df['Title'].value_counts()
print("\nDistribution des classes après suréchantillonnage :")
print(class_distribution_after.head())


Distribution des classes avant suréchantillonnage :
Title
7     31
0     10
25    10
8     10
12     9
Name: count, dtype: int64

Distribution des classes après suréchantillonnage :
Title
2     32
8     32
15    32
4     32
11    32
Name: count, dtype: int64


## Legal BERT V2

In [8]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# Charger le dataset
df = resampled_df

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(df['Content'], df['Title'], test_size=0.2, random_state=42)

# Charger le tokenizer de LegalBERT
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased')

# Tokenisation des données
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512)

train_encodings = tokenize_function(X_train.tolist())
test_encodings = tokenize_function(X_test.tolist())

# Créer un Dataset personnalisé pour BERT
class LegalBERTDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

# Créer les datasets pour l'entraînement et l'évaluation
train_dataset = LegalBERTDataset(train_encodings, y_train)
test_dataset = LegalBERTDataset(test_encodings, y_test)

# Charger le modèle pré-entraîné de LegalBERT pour la classification
model = BertForSequenceClassification.from_pretrained('nlpaueb/legal-bert-base-uncased', num_labels=4)

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',          # Répertoire de sortie
    evaluation_strategy="epoch",     # Stratégie d'évaluation par époque
    learning_rate=2e-5,              # Taux d'apprentissage
    per_device_train_batch_size=8,   # Taille du batch pour l'entraînement
    per_device_eval_batch_size=8,    # Taille du batch pour l'évaluation
    num_train_epochs=5,              # Nombre d'époques
    weight_decay=0.01,               # Décroissance du poids
    logging_dir='./logs',            # Répertoire des logs
)

# Initialiser le Trainer
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Entraîner le modèle
trainer.train()

# Évaluer le modèle
trainer.evaluate()

# Prédictions sur le jeu de test
predictions = trainer.predict(test_dataset)

# Afficher les résultats des prédictions
print(predictions.predictions.argmax(axis=-1))

# Sauvegarder le modèle et le tokenizer
model.save_pretrained('./legal_bert_model')
tokenizer.save_pretrained('./legal_bert_model')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/435 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

{'eval_loss': 0.0882098600268364, 'eval_runtime': 17.327, 'eval_samples_per_second': 9.984, 'eval_steps_per_second': 1.27, 'epoch': 1.0}


  0%|          | 0/22 [00:00<?, ?it/s]

...

  0%|          | 0/22 [00:00<?, ?it/s]

{'eval_loss': 0.08310308307409286, 'eval_runtime': 14.4375, 'eval_samples_per_second': 11.983, 'eval_steps_per_second': 1.524, 'epoch': 3.0}


  0%|          | 0/22 [00:00<?, ?it/s]

{'eval_loss': 0.06717950105667114, 'eval_runtime': 15.1776, 'eval_samples_per_second': 11.398, 'eval_steps_per_second': 1.449, 'epoch': 4.0}


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 774546176 vs 774546064

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.metrics import accuracy_score, confusion_matrix

# Charger le dataset
df = pdf_dataframe

# Charger le modèle et le tokenizer sauvegardés
model = BertForSequenceClassification.from_pretrained('./legal_bert_model')
tokenizer = BertTokenizer.from_pretrained('./legal_bert_model')

# Fonction pour prédire la classe d'un texte
def predict(texts):
    # Tokeniser les nouveaux textes
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    
    # Obtenir les prédictions du modèle
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
    
    # Obtenir les indices des classes prédominantes
    predictions = torch.argmax(logits, dim=-1)
    return predictions

# Tester le modèle avec le contenu du dataset
new_texts = df['Content'].tolist()  # Liste des contenus des PDFs
predicted_labels = predict(new_texts)

# Ajouter les prédictions dans le dataframe
df['Predicted_Label'] = predicted_labels.numpy()

# Afficher les résultats avec les titres
print("Exemples de prédictions comparées aux véritables titres :")
print(df[['Title', 'Predicted_Label']].head(20))

# Calcul de la précision
accuracy = accuracy_score(df['Title'], df['Predicted_Label'])
print(f"Précision du modèle : {accuracy * 100:.2f}%")

# Confusion Matrix pour visualiser les bonnes et mauvaises prédictions
conf_matrix = confusion_matrix(df['Title'], df['Predicted_Label'])
print("\nMatrice de confusion :")
print(conf_matrix)

# Nombre de bonnes réponses
correct_predictions = (df['Title'] == df['Predicted_Label']).sum()
print(f"\nNombre de bonnes réponses : {correct_predictions}")

# Nombre de mauvaises réponses
incorrect_predictions = len(df) - correct_predictions
print(f"Nombre de mauvaises réponses : {incorrect_predictions}")


Exemples de prédictions comparées aux véritables titres :
   Title  Predicted_Label
0      0                0
1      1                1
2      0                0
3      2                2
4      0                0
Précision du modèle : 16.00%

Matrice de confusion :
[[10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [31  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0]
 [ 0  0 10  0  0  0  0  0  0  0  0  0  0  0  